Main measures: Binary, discrete measure using main class 

Attribute industry to the first listed class in patent

In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import design_class_dictionary

%matplotlib inline

In [19]:
master = pd.read_csv('data/master_503107.csv')
d2d_seen = pd.read_csv('data/final_main_d2d_seen.csv')
d2d_seen_cont = pd.read_csv('data/final_main_d2d_seen_cont.csv')
d2u_seen = pd.read_csv('data/final_main_d2u_seen.csv')
d2u_seen_cont = pd.read_csv('data/final_main_d2u_seen_cont.csv')

# Correlation of discrete to continuous

First assign focal class to patents

In [22]:
patent_classifications = pd.read_csv('data/uspc_current.tsv', delimiter = '\t', usecols=['patent_id','mainclass_id','sequence'], dtype={'patent_id':str, 'mainclass_id':str})
patent_classifications.head()

,patent_id,mainclass_id,sequence
0,3930271,2,0
1,3930271,2,1
2,3930271,2,2
3,3930271,473,3
4,3930272,5,0


In [28]:
listed_patents = list(set(master.patent_number.tolist()))
patent_classifications = patent_classifications.loc[patent_classifications['patent_id'].isin(listed_patents)]

#select all first classifications
patent_classifications = patent_classifications.loc[patent_classifications.sequence == 0]
patent_classifications = patent_classifications.rename(index=str, columns={'patent_id':'patent_number'})
patent_classifications.head()

,patent_number,mainclass_id,sequence
21969998,D257752,D19,0
21970267,D257924,D06,0
21970995,D258382,D23,0
21970996,D258383,D23,0
21971443,D258678,D23,0


A patent is considered novel if ANY of it's backwards combinations is first seen in the discrete case

In [33]:
d2d_seen.head()

,patent_number,priority_date,cite_mainclass,focal_mainclass,first_seen
0,D466542,1902,D16,D16,1
1,D499408,1904,D14,D14,1
2,D510547,1905,D12,D12,1
3,D523461,1906,D16,D16,0
4,D523461,1906,D14,D16,1


In [77]:
#design to design
d2d = d2d_seen.groupby('patent_number').apply(lambda x: np.any(x['first_seen']).astype(int))\
.reset_index().rename(index=int, columns={0:'is_novel'})

d2d.head()

,patent_number,is_novel
0,D258382,0
1,D258383,0
2,D258678,0
3,D258755,0
4,D258990,1


In [105]:
#design to utility
d2u = d2u_seen.groupby('patent_number').apply(lambda x: np.any(x['first_seen']).astype(int))\
.reset_index().rename(index=int, columns={0:'is_novel'})

d2u.head()

,patent_number,is_novel
0,D257752,1
1,D257924,0
2,D258766,1
3,D259281,1
4,D259509,0


apply priority date and focal mainclass

In [90]:
#design to design
date = d2d_seen.groupby('patent_number')['priority_date'].mean().reset_index()\
.rename(index=int, columns={0:'priority_date'})

d2d = pd.merge(d2d, date, on='patent_number')
d2d = pd.merge(d2d, patent_classifications, on='patent_number')

d2d.head()

,patent_number,is_novel,priority_date,mainclass_id,sequence
0,D258382,0,1980,D23,0
1,D258383,0,1980,D23,0
2,D258678,0,1980,D23,0
3,D258755,0,1980,D23,0
4,D258990,1,1980,D25,0


In [106]:
#design to utility
date = d2u_seen.groupby('patent_number')['priority_date'].mean().reset_index()\
.rename(index=int, columns={0:'priority_date'})

d2u = pd.merge(d2u, date, on='patent_number')
d2u = pd.merge(d2u, patent_classifications, on='patent_number')

d2u.head()

,patent_number,is_novel,priority_date,mainclass_id,sequence
0,D257752,1,1980,D19,0
1,D257924,0,1980,D06,0
2,D258766,1,1980,D25,0
3,D259281,1,1980,D25,0
4,D259509,0,1980,D24,0


Calculate max novelty for continuous measure

In [91]:
d2d_seen_cont.head()

,patent_number,priority_date,cite_mainclass,focal_mainclass,first_seen,Nt,Nijt,Nit,Nij,commonness
0,D258382,1980,D23,D23,0,3970,200,598,598,2.220333
1,D258383,1980,D23,D23,0,3970,200,598,598,2.220333
2,D258678,1980,D23,D23,0,3970,200,598,598,2.220333
3,D258755,1980,D23,D23,0,3970,200,598,598,2.220333
4,D258990,1980,D25,D25,0,3970,78,289,289,3.707571


In [92]:
def commonness_to_novelty(df):
    df['novelty'] = np.log(df['commonness'])*(-1)
    return df

In [95]:
d2d_seen_cont = commonness_to_novelty(d2d_seen_cont)
d2d_seen_cont.head()

,patent_number,priority_date,cite_mainclass,focal_mainclass,first_seen,Nt,Nijt,Nit,Nij,commonness,novelty
0,D258382,1980,D23,D23,0,3970,200,598,598,2.220333,-0.797657
1,D258383,1980,D23,D23,0,3970,200,598,598,2.220333,-0.797657
2,D258678,1980,D23,D23,0,3970,200,598,598,2.220333,-0.797657
3,D258755,1980,D23,D23,0,3970,200,598,598,2.220333,-0.797657
4,D258990,1980,D25,D25,0,3970,78,289,289,3.707571,-1.310377


In [107]:
d2u_seen_cont = commonness_to_novelty(d2u_seen_cont)
d2u_seen_cont.head()

,patent_number,priority_date,cite_mainclass,focal_mainclass,first_seen,Nt,Nijt,Nit,Nij,commonness,novelty
0,D257752,1980,211,D19,0,4649,10,100,93,4.998925,-1.609223
1,D257924,1980,211,D06,0,4649,39,345,93,5.650958,-1.731825
2,D258766,1980,52,D25,0,4649,76,234,137,11.021399,-2.399839
3,D258766,1980,40,D25,0,4649,1,234,53,0.374859,0.981206
4,D258766,1980,49,D25,0,4649,24,234,42,11.352869,-2.429471


Take max of novelty of each patent

In [100]:
#design to design
max_novelty = d2d_seen_cont.groupby('patent_number')['novelty'].max().reset_index()\
.rename(index=str, columns={'novelty':'max_novelty'})
max_novelty.head()

,patent_number,max_novelty
0,D258382,-0.797657
1,D258383,-0.797657
2,D258678,-0.797657
3,D258755,-0.797657
4,D258990,0.809274


In [103]:
d2d = pd.merge(d2d, max_novelty, on='patent_number')
d2d.head()

,patent_number,is_novel,priority_date,mainclass_id,sequence,max_novelty
0,D258382,0,1980,D23,0,-0.797657
1,D258383,0,1980,D23,0,-0.797657
2,D258678,0,1980,D23,0,-0.797657
3,D258755,0,1980,D23,0,-0.797657
4,D258990,1,1980,D25,0,0.809274


In [108]:
#design to utility
max_novelty = d2u_seen_cont.groupby('patent_number')['novelty'].max().reset_index()\
.rename(index=str, columns={'novelty':'max_novelty'})
# max_novelty.head()
d2u = pd.merge(d2u, max_novelty, on='patent_number')
d2u.head()

,patent_number,is_novel,priority_date,mainclass_id,sequence,max_novelty
0,D257752,1,1980,D19,0,-1.609223
1,D257924,0,1980,D06,0,-1.731825
2,D258766,1,1980,D25,0,0.981206
3,D259281,1,1980,D25,0,-2.399839
4,D259509,0,1980,D24,0,-1.578738


# Compile 4 measures together by industry

In [110]:
#design to design
d2d.head()

,patent_number,is_novel,priority_date,mainclass_id,sequence,max_novelty
0,D258382,0,1980,D23,0,-0.797657
1,D258383,0,1980,D23,0,-0.797657
2,D258678,0,1980,D23,0,-0.797657
3,D258755,0,1980,D23,0,-0.797657
4,D258990,1,1980,D25,0,0.809274


In [113]:
d2d.sort_values('mainclass_id').mainclass_id.unique()

array(['1', '24', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08',
       'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17',
       'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26',
       'D27', 'D28', 'D29', 'D30', 'D32', 'D34', 'D6', 'D7', 'D9', 'D99'],
      dtype=object)

Remove any non design classes and harmonize D6, D7, D9

In [123]:
d2d['mainclass_id'] = d2d['mainclass_id'].replace(design_class_dictionary.class_dictionary)
d2d = d2d.loc[d2d.mainclass_id.str.contains('D')]

In [140]:
col1 = d2d.groupby('mainclass_id').apply(lambda x: np.count_nonzero(x['is_novel'])).reset_index()\
.rename(index=int, columns={0:'D2D_discrete_novelty_count'})

col1 = col1.append(col1.sum(), ignore_index=True)

col1.iloc[33,0] = 'total'

In [180]:
col2 = d2d.groupby('mainclass_id')['max_novelty'].mean().reset_index()\
.rename(index=str, columns={'max_novelty':'D2D_continuous_mean_max_novelty'})

col2 = col2.append(col2.mean(), ignore_index=True)

col2.iloc[33,0] = 'total'

In [153]:
#design to utility
np.sort(d2u.mainclass_id.unique())

array(['70', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08',
       'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17',
       'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26',
       'D27', 'D28', 'D29', 'D30', 'D32', 'D34', 'D6', 'D7', 'D8', 'D9',
       'D99'], dtype=object)

Remove non design classes and harmonize non standard classes

In [154]:
d2u['mainclass_id'] = d2u['mainclass_id'].replace(design_class_dictionary.class_dictionary)
d2u = d2u.loc[d2u.mainclass_id.str.contains('D')]

In [182]:
col3 = d2u.groupby('mainclass_id').apply(lambda x: np.count_nonzero(x['is_novel'])).reset_index()\
.rename(index=int, columns={0:'D2U_discrete_novelty_count'})

col3 = col3.append(col3.sum(), ignore_index=True)

col3.iloc[33,0] = 'total'

In [184]:
col4 = d2u.groupby('mainclass_id')['max_novelty'].mean().reset_index()\
.rename(index=str, columns={'max_novelty':'D2U_continuous_mean_max_novelty'})

col4 = col4.append(col4.mean(), ignore_index=True)

col4.iloc[33,0] = 'total'

In [189]:
measures = pd.merge(col1, col2)
measures = pd.merge(measures, col3)
measures = pd.merge(measures, col4)
measures['class_name'] = measures['mainclass_id'].map(design_class_dictionary.class_labels)
measures

,mainclass_id,D2D_discrete_novelty_count,D2D_continuous_mean_max_novelty,D2U_discrete_novelty_count,D2U_continuous_mean_max_novelty,class_name
0,D01,12,0.506282,162,-1.603018,Edible Products
1,D02,38,-0.894670,296,-1.753094,Apparel and Haberdashery
2,D03,46,0.871490,331,-0.150299,"Travel Goods, Personal Belongings, and Storage..."
3,D04,25,-0.727622,168,-1.154202,Brushware
4,D05,16,0.018399,193,-0.835168,Textile or Paper Yard Goods; Sheet Material
5,D06,70,0.619925,444,-0.340744,Furnishings
6,D07,42,0.654620,358,-0.312512,Equipment for Preparing or Serving Food or Dri...
7,D08,32,0.653117,556,-0.249547,Tools and Hardware
8,D09,46,0.754233,356,-0.312400,Packages and Containers for Goods
9,D10,37,0.241391,452,-0.589155,"Measuring, Testing or Signaling Instruments"


Correlations between discrete and continuous

In [198]:
correlations = measures.iloc[0:32,1:5].corr()

save the data

In [199]:
measures.to_csv('data/corr_table_cont_disc.csv')
correlations.to_csv('data/corr_matrix_cont_disc.csv')